In [1]:
import re

In [47]:
class Parser():
    
    def __init__(self,input_file = 'test_input.txt'):
        self.file = open(input_file,'r')
        
    current_cmd = None

    def advance(self):
        self.current_cmd = ''
        # Read next command and make it the current command
        while (self.current_cmd ==''):
            line = self.file.readline()
            cmt = re.search('//',line)
            if (cmt == None):
                # Non-commented
                self.current_cmd = line.strip()
            else:
                # Commented
                cmd = line[:cmt.start()].strip()
                self.current_cmd = cmd
                
        print(self.current_cmd)

    def commandType(self):
        current_cmd = self.current_cmd.copy()
        cmd = current_cmd.split(' ')[0]
        if (cmd =='push'):
            return 'C_PUSH'
        elif (cmd=='pop'):
            return 'C_POP'
        else :
            return 'C_ARITHMETIC'
        
        
    def arg1(self):
        current_cmd = self.current_cmd.copy()
        comp = current_cmd.split(' ')
        if (len(comp)==3):
            # C_PUSH or C_POP
            return comp[1]
        else:
            # C_ARITHMETIC/LOGICAL
            return comp[0]
            
            
    def arg2(self):
    # Called only if non arithmetic command
        current_cmd = self.current_cmd.copy()
        comp = current_cmd.split(' ')
        return int(comp[2])

p = Parser()

In [49]:
p.advance()

456


In [53]:
class CodeWriter:
    def __init__(self,output_file):
        self.f = open(output_file,'w+')
        
    def close(self):
        self.f.close()
        
###########################  Helper Functions ########################################
    def segPtr(self,segment):
        switcher={
                'local':'LCL',
                'argument':'ARG',
                'this':'THIS',
                'that':'THAT',
             }
        return switcher.get(segment,"Invalid")
    
    def pointer(self,index):
        switcher={
                 0:'THIS',
                 1:'THAT'
             }
        return switcher.get(index,"Invalid")
    
    
    
###########################  PUSH/POP ########################################
    def writePushPop(self,command,segment,index):
        if command == 'C_PUSH':
            if (segment == 'local' or segment == 'argument' or segment == 'this' or segment == 'that'):
                # Command
                l0 = '// push ' + segment + ' ' + str(index) + '\n'
                
                # addr = segmentPtr+index
                l1 = '@' + self.segPtr(segment) + '\n'
                l2 = 'D = M\n'
                l3 = '@' + str(index) + '\n'
                l4 = 'D = D + A\n'
                l5 = '@addr\n' 
                l6 = 'M = D\n'
                # *SP = *addr
                l7 = 'A = M\n'
                l8 = 'D = M\n'
                l9 = '@SP\n'
                l10 = 'A = M\n'             
                l11 = 'M = D\n'
                # SP++
                l12 = '@SP\n'
                l13 = 'M = M + 1\n'
                self.f.writelines([l0,l1,l2,l3,l4,l5,l6,l7,l8,l9,l10,l11,l12,l13])

            elif (segment == 'constant'):
                 # Command
                l0 = '// push ' + segment + ' ' + str(index) + '\n'
                 # *SP = index
                l1 = '@' + str(index) + '\n'
                l2 = 'D = A\n'
                l3 = '@SP\n'
                l4 = 'A = M\n'
                l5 = 'M = D\n' 
                # SP++
                l6 = '@SP\n'
                l7 = 'M = M + 1\n'
                self.f.writelines([l0,l1,l2,l3,l4,l5,l6,l7])

            elif (segment == 'static'):
                # Command
                l0 = '// push ' + segment + ' ' + str(index) + '\n'
                 # *SP = Filename.index
                l1 = '@Filename.' + str(index) + '\n'
                l2 = 'D = M\n'
                l3 = '@SP\n'
                l4 = 'A = M\n'
                l5 = 'M = D\n' 
                # SP++
                l6 = '@SP\n'
                l7 = 'M = M + 1\n'
                self.f.writelines([l0,l1,l2,l3,l4,l5,l6,l7])

            elif (segment == 'temp'):
                l0 = '// push ' + segment + ' ' + str(index) + '\n'
                # addr = 5 +index
                l1 = '@5\n'
                l2 = 'D = M\n'
                l3 = '@' + str(index) + '\n'
                l4 = 'D = D + A\n'
                l5 = '@addr\n' 
                l6 = 'M = D\n'
                # *SP = *addr
                l7 = 'A = M\n'
                l8 = 'D = M\n'
                l9 = '@SP\n'
                l10 = 'A = M\n'             
                l11 = 'M = D\n'
                # SP++
                l12 = '@SP\n'
                l13 = 'M = M + 1\n'
                self.f.writelines([l0,l1,l2,l3,l4,l5,l6,l7,l8,l9,l10,l11,l12,l13])

            elif (segment == 'pointer'):
                # Command
                l0 = '// push ' + segment + ' ' + str(index) + '\n'
                 # *SP = THIS/THAT
                l1 = '@' + self.pointer(index) + '\n'
                l2 = 'D = M\n'
                l3 = '@SP\n'
                l4 = 'A = M\n'
                l5 = 'M = D\n' 
                # SP++
                l6 = '@SP\n'
                l7 = 'M = M + 1\n'
                self.f.writelines([l0,l1,l2,l3,l4,l5,l6,l7])
                
        elif command == 'C_POP':
            if (segment == 'local' or segment == 'argument' or segment == 'this' or segment == 'that'):
                # Command
                l0 = '// pop ' + segment + ' ' + str(index) + '\n'
                # addr = segmentPtr+index
                l1 = '@' + self.segPtr(segment) + '\n'
                l2 = 'D = M\n'
                l3 = '@' + str(index) + '\n'
                l4 = 'D = D + A\n'
                l5 = '@addr\n' 
                l6 = 'M = D\n'
                # SP--
                l7 = '@SP\n'
                l8 = 'M = M-1\n'
                # *addr = *SP
                l9 = '@SP\n'
                l10 = 'A = M\n'             
                l11 = 'D = M\n'
                l12 = '@addr\n'
                l13 = 'A = M\n'
                l14 = 'M = D\n'
                self.f.writelines([l0,l1,l2,l3,l4,l5,l6,l7,l8,l9,l10,l11,l12,l13,l14])               
                
            elif (segment == 'static'):
                # Command
                l0 = '// pop ' + segment + ' ' + str(index) + '\n'
                # SP--
                l1 = '@SP\n'
                l2 = 'M = M - 1\n'
                # Filename.index = *SP
                l3 = '@SP\n'
                l4 = 'A = M\n'
                l5 = 'D = M\n' 
                l6 = '@Filename.' + str(index) + '\n'
                l7 = 'M = D\n'
                self.f.writelines([l0,l1,l2,l3,l4,l5,l6,l7])
                
            elif (segment == 'temp'):
                 # Command
                l0 = '// pop ' + segment + ' ' + str(index) + '\n'
                # addr = 5+index
                l1 = '@5\n'
                l2 = 'D = M\n'
                l3 = '@' + str(index) + '\n'
                l4 = 'D = D + A\n'
                l5 = '@addr\n' 
                l6 = 'M = D\n'
                # SP--
                l7 = '@SP\n'
                l8 = 'M = M-1\n'
                # *addr = *SP
                l9 = '@SP\n'
                l10 = 'A = M\n'             
                l11 = 'D = M\n'
                l12 = '@addr\n'
                l13 = 'A = M\n'
                l14 = 'M = D\n'
                self.f.writelines([l0,l1,l2,l3,l4,l5,l6,l7,l8,l9,l10,l11,l12,l13,l14])   
                
            elif (segment == 'pointer'):
                # Command
                l0 = '// pop ' + segment + ' ' + str(index) + '\n'
                # SP--
                l1 = '@SP\n'
                l2 = 'M = M - 1\n'
                # THIS/THAT = *SP
                l3 = '@SP\n'
                l4 = 'A = M\n'
                l5 = 'D = M\n' 
                l6 = '@' + self.pointer(index) + '\n'
                l7 = 'M = D\n'
                self.f.writelines([l0,l1,l2,l3,l4,l5,l6,l7])
                
                
                
###########################  ARITHMETIC ########################################              
    def writeArithmetic(self,operation):
        if (operation=='add'):
            # Command                
            l0 = '// add\n'
            # SP=SP-2
            l1 = '@2\n'
            l2 = 'D = A\n'
            l3 = '@SP\n'
            l4 = 'M = M - D\n'
            # D = *SP+*(SP+1)
            l5 = 'A = M\n'
            l6 = 'D = M\n'
            l7 = 'A = A + 1\n'
            l8 = 'D = D + M\n'
            #*SP = D
            l9 = '@SP\n'
            l10 = 'A = M\n'
            l11 = 'M = D\n'
            # SP++
            l12 = '@SP\n'
            l13 = 'M = M + 1\n'
        elif (operation=='sub'):
            # Command                
            l0 = '// add\n'
            # SP=SP-2
            l1 = '@2\n'
            l2 = 'D = A\n'
            l3 = '@SP\n'
            l4 = 'M = M - D\n'
            # D = *SP-*(SP+1)
            l5 = 'A = M\n'
            l6 = 'D = M\n'
            l7 = 'A = A + 1\n'
            l8 = 'D = D - M\n'
            #*SP = D
            l9 = '@SP\n'
            l10 = 'A = M\n'
            l11 = 'M = D\n'
            # SP++
            l12 = '@SP\n'
            l13 = 'M = M + 1\n'
        elif (operation=='eq'):
            # Command                
            l0 = '// add\n'
            # SP=SP-2
            l1 = '@2\n'
            l2 = 'D = A\n'
            l3 = '@SP\n'
            l4 = 'M = M - D\n'
            # D = (*SP-*(SP+1))
            l5 = 'A = M\n'
            l6 = 'D = M\n'
            l7 = 'A = A + 1\n'
            l8 = 'D = D - M\n'
            # D == 0
            l9 = '@EQUAL'
            l10 = 'D; JEQ'
            # defining EQUAL
            l11 = '(EQUAL)'
            l12 = ''
            #*SP = D
            l9 = '@SP\n'
            l10 = 'A = M\n'
            l11 = 'M = D\n'
            # SP++
            l12 = '@SP\n'
            l13 = 'M = M + 1\n'





In [54]:
c = CodeWriter('output.txt')

In [55]:
c.writePushPop('C_POP','pointer',1)
c.close()

In [ ]:
class Main():
    def __init__()